In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta

In [2]:
def nofluffjobs_page_job_offers() -> list:

    nofluffjobs_list = list()
    

    url = "https://nofluffjobs.com/pl/praca-zdalna/python?criteria=city%3Dwarszawa%20seniority%3Dtrainee,junior%20%20salary<pln12000m&page=1"
   
    flag = True
    try:
        page = requests.get(url)
    except requests.exceptions.ConnectionError as err:
        flag = False
    if flag:
        soup = BeautifulSoup(page.content, "html.parser")
        
        for job_element in soup.select('a[class*="posting-list-item posting-list-item--"]'):
            
            
            nofluffjobs_dict = dict()
            company_element = job_element.find("span", class_="d-block posting-title__company text-truncate")
            job_title_element = job_element.find("h3", class_="posting-title__position text-truncate color-main ng-star-inserted")
            if "Fullstack" not in job_title_element.text and "DevOps" not in job_title_element.text and "Golang" not in job_title_element.text:
                link_url = "https://nofluffjobs.com" + job_element["href"]
                pattern = re.compile(r'https?://([\w.\.\-]+)')
                website_name = pattern.match(link_url)
                page_job_element = requests.get(link_url)
                soup_page = BeautifulSoup(page_job_element.content, "html.parser")
                position = soup_page.find("span", class_="mr-10 font-weight-medium")
                publication_date_element = soup_page.find("div", class_="posting-time-row")
                
                days_after_publication = re.findall(r'\b\d+\b', publication_date_element.text.strip())
                if len(days_after_publication) == 0:
                    publication_date = datetime.today().strftime("%Y-%m-%d")
                
                else:
                    publication_date = (datetime.today() - (timedelta(days=int(days_after_publication[0])))).strftime("%Y-%m-%d")
                        
                nofluffjobs_dict["publication_date"] = publication_date
                nofluffjobs_dict["company"] = company_element.text.strip()
                nofluffjobs_dict["title"] = job_title_element.text.strip()
                nofluffjobs_dict["position"] = position.text.strip()
                nofluffjobs_dict["link_url"] = link_url
                nofluffjobs_dict["website"] = website_name[0]

                nofluffjobs_list.append(nofluffjobs_dict)
    return nofluffjobs_list


In [3]:
import pandas as pd
import numpy as np

nofluffjobs_list = nofluffjobs_page_job_offers()
df_raw = pd.DataFrame.from_records(nofluffjobs_list)
df = df_raw.copy()
df['publication_date'] = pd.to_datetime(df['publication_date'], infer_datetime_format=True)
df.drop_duplicates(subset=['publication_date', 'company', 'title'], inplace=True, ignore_index=True)




## WSTAW SOBIE TU TEKST

In [4]:
df

,publication_date,company,title,position,link_url,website
0,2022-08-25,Devire,Python Developer (Data Science),Junior,https://nofluffjobs.com/pl/job/python-develope...,https://nofluffjobs.com
1,2022-08-18,Amartus Polska Sp. z o.o.,Junior Phyton Developer,Junior,https://nofluffjobs.com/pl/job/junior-phyton-d...,https://nofluffjobs.com
2,2022-08-18,IDEA team within Narodowe Centrum Badań Jądrowych,Junior Python Developer,Junior,https://nofluffjobs.com/pl/job/junior-python-d...,https://nofluffjobs.com
3,2022-08-26,Webinterpret,Junior Data Scientist,Junior,https://nofluffjobs.com/pl/job/junior-data-sci...,https://nofluffjobs.com
4,2022-08-24,Formeld Poland,Remote Junior Test Engineer with German,Junior,https://nofluffjobs.com/pl/job/remote-junior-t...,https://nofluffjobs.com
5,2022-08-23,Sport Vision Technology sp. z o.o. sp. k.,Junior AI Developer,Junior,https://nofluffjobs.com/pl/job/junior-ai-devel...,https://nofluffjobs.com
6,2022-08-20,Deloitte Advisory,AI LAB,Trainee,https://nofluffjobs.com/pl/job/ai-lab-deloitte...,https://nofluffjobs.com
7,2022-08-27,Astrotectonic,Junior Data Scientist,Junior,https://nofluffjobs.com/pl/job/junior-data-sci...,https://nofluffjobs.com


In [6]:
df.columns

Index(['publication_date', 'company', 'title', 'position', 'link_url',
       'website'],
      dtype='object')